## Deze is niet afgemaakt, werkt niet

# Imports

In [2]:
import re
import pandas as pd
import numpy as np

# Data

In [3]:
bestand = r'.\data\\voorbeeld6.txt'

with open(bestand, 'r') as file:
    data = (file
            .read()
             .splitlines()
            )
    
df = (
    pd.DataFrame(data)[0]
    .str.split('', expand=True)
    .replace('', np.nan)
    .dropna(axis=1)
    .reset_index(drop=True)
)


# Dag 6.1

In [4]:
def find_initial_coordinate(df):
    indeces = np.nonzero(df.isin(['^', 'v', '>', '<']))
    row_index = indeces[0][0]
    col_index = indeces[1][0]
    inital_coordinates = (row_index, col_index)
    return inital_coordinates

In [5]:
def determine_look_direction(df, coordinates):
    row_index = coordinates[0]
    col_index = coordinates[1]
    if df.iloc[row_index, col_index] == '^':
        direction_looking = 'up'

    elif df.iloc[row_index, col_index] == '>':
        direction_looking = 'right'

    elif df.iloc[row_index, col_index] == 'v':
        direction_looking = 'down'

    elif df.iloc[row_index, col_index] == '<':
        direction_looking = 'left'

    else:
        raise Exception('coordinates are wrong, no looking position')
    
    new_df = df.copy()
    new_df.iloc[row_index, col_index] = '.'

    return new_df, direction_looking

In [6]:
def found_the_escape(df, coordinates, direction_looking):
    last_index_col = df.shape[1]-1
    last_index_row = df.shape[0]-1
    found_escape = False

    if (coordinates[0]==0) & (direction_looking == 'up'):
        found_escape = True
    
    if (coordinates[0]==last_index_row) & (direction_looking == 'down'):
        found_escape = True

    if (coordinates[1]==0) & (direction_looking == 'left'):
        found_escape = True
    
    if (coordinates[1]==last_index_col) & (direction_looking == 'right'):
        found_escape = True
    
    return found_escape
    

    



In [7]:
def define_direction(direction_looking):
    row_direction = 0
    col_direction = 0
    if direction_looking == 'up':
        row_direction = -1
    elif direction_looking == 'down':
        row_direction = 1
    elif direction_looking == 'right':
        col_direction = 1
    elif direction_looking == 'left':
        col_direction = -1

    return row_direction, col_direction

In [8]:
max(df.iloc[:6, 4].str.find('X'))

-1

In [44]:
def walk_as_far_as_posible(
        df, 
        standing_ad_cooridinates, 
        direction_looking, 
        visits_facing_up, 
        visits_facing_down, 
        visits_facing_right, 
        visits_facing_left, 
        visits):

    in_loop = False

    row_index = standing_ad_cooridinates[0]
    col_index = standing_ad_cooridinates[1]

    found_escape = False
    while (found_escape == False) & (in_loop == False):
        if direction_looking == 'up':
            serie_to_walk = df.iloc[:row_index, col_index]
            try:
                first_stop = max(np.nonzero(serie_to_walk.isin(["#"]))[0])
                if max(visits_facing_up.iloc[first_stop+1:row_index, col_index].str.find("X")) > -1:
                    in_loop = True
                    break
                visits_facing_up.iloc[first_stop+1:row_index, col_index] = 'X'
                visits.iloc[first_stop+1:row_index, col_index] = 'X'
                direction_looking = "right"
                row_index = first_stop + 1
            except: # no boundary
                visits.iloc[:row_index, col_index] = 'X'
                found_escape = True

        if direction_looking == "right":
            serie_to_walk = df.iloc[row_index, col_index:]
            try:
                first_stop = min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + col_index
                if max(visits_facing_right.iloc[row_index, col_index:first_stop].str.find("X")) > -1:
                    in_loop = True
                    break
                visits_facing_right.iloc[row_index, col_index:first_stop] = 'X'
                visits.iloc[row_index, col_index:first_stop] = 'X'
                direction_looking = "down"
                col_index = first_stop - 1
            except:
                visits.iloc[row_index, col_index:] = 'X'
                found_escape = True
        
        if direction_looking == "down":
            serie_to_walk = df.iloc[row_index:, col_index]
            try:
                first_stop =  min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + row_index
                if max(visits_facing_down.iloc[row_index:first_stop, col_index].str.find("X")) > -1:
                    in_loop = True
                    break
                visits_facing_down.iloc[row_index:first_stop, col_index] = 'X'
                visits.iloc[row_index:first_stop, col_index] = 'X'
                direction_looking = "left"
                row_index = first_stop - 1
            except:
                visits.iloc[row_index:, col_index] = 'X'
                found_escape = True
        
        if direction_looking == "left":
            serie_to_walk = df.iloc[row_index, :col_index]
            try:
                first_stop =  max(np.nonzero(serie_to_walk.isin(["#"]))[0]) 
                if max(visits_facing_left.iloc[row_index, first_stop+1:col_index].str.find("X")) > -1:
                    in_loop = True
                    break
                visits_facing_left.iloc[row_index, first_stop+1:col_index] = 'X'
                visits.iloc[row_index, first_stop+1:col_index] = 'X'
                direction_looking = "up"
                col_index = first_stop + 1
            except:
                visits.iloc[row_index, :col_index] = 'X'
                found_escape = True

    return visits, in_loop



In [45]:
def make_visits_including_facing(visits, direction_looking, coordinates):
    
    visits_facing_up = visits.copy()
    visits_facing_down = visits.copy()
    visits_facing_right = visits.copy()
    visits_facing_left = visits.copy()
    
    if direction_looking == 'up':
        visits_facing_up.iloc[coordinates[0], coordinates[1]] = 'X'
    elif direction_looking == 'down':
        visits_facing_down.iloc[coordinates[0], coordinates[1]]  = 'X'
    elif direction_looking == 'right':
        visits_facing_right.iloc[coordinates[0], coordinates[1]] = 'X'
    elif direction_looking == 'left':
        visits_facing_left.iloc[coordinates[0], coordinates[1]] = 'X'

       
    return visits_facing_up, visits_facing_down, visits_facing_right, visits_facing_left

In [46]:
def walk_and_count(df):
    standing_ad_cooridinates = find_initial_coordinate(df)
    df, direction_looking = determine_look_direction(df, standing_ad_cooridinates)
    visits = pd.DataFrame(columns=df.columns, index=df.index)
    visits_facing_up, visits_facing_down, visits_facing_right, visits_facing_left = make_visits_including_facing(visits, direction_looking, standing_ad_cooridinates)
    visits.iloc[standing_ad_cooridinates[0], standing_ad_cooridinates[1]] = 'X'

    visits, in_loop = (
        walk_as_far_as_posible(
                df, 
                standing_ad_cooridinates, 
                direction_looking, 
                visits_facing_up, visits_facing_down, visits_facing_right, visits_facing_left,
                visits
                )
    )
    
    if in_loop == False:
        number_of_visit_places = (visits == 'X').sum().sum()
    return number_of_visit_places, in_loop, visits

In [47]:
number_of_visit_places, in_loop, visits_part1 = walk_and_count(df)
number_of_visit_places

IndexError: index 0 is out of bounds for axis 0 with size 0

# Dag 6.2

In [13]:
def make_new_data(df,row_index, col_index):
    new_df = df.copy()
    possible = True
    if len(re.findall("[<>v^#]", df.iloc[row_index, col_index]))>0:
           possible = False
    else: 
       new_df.iloc[row_index, col_index] = '#'
    
    return new_df, possible

In [14]:
def count_possible_obstakels(data, visits_par1):
    total_inloop = 0
    list_of_visits = np.nonzero(visits_part1.isin(["X"]))
    i = 0
    for i in range(len(list_of_visits[0])):
        print(i)
        row_index = list_of_visits[0][i]
        col_index = list_of_visits[1][i]

        new_df, possible = make_new_data(data, row_index, col_index)
        print(row_index)
        print(col_index)
        print(possible)
        if possible:
            number_of_visits, in_loop, list_of_visits_ = walk_and_count(new_df)
            if in_loop:
                total_inloop += 1
        
    return total_inloop   

In [15]:
# count_possible_obstakels(df, visits_part1)

In [16]:
new_df, possible = make_new_data(df, 6, 3)
new_df

,1,2,3,4,5,6,7,8,9,10
0,.,.,.,.,#,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,#
2,.,.,.,.,.,.,.,.,.,.
3,.,.,#,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,#,.,.
5,.,.,.,.,.,.,.,.,.,.
6,.,#,.,#,^,.,.,.,.,.
7,.,.,.,.,.,.,.,.,#,.
8,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,.,#,.,.,.


'up'

In [57]:
df, direction_looking = determine_look_direction(new_df, (6,4))
visits = pd.DataFrame(columns=df.columns, index=df.index)
visits_facing_up, visits_facing_down, visits_facing_right, visits_facing_left = make_visits_including_facing(visits, direction_looking, (6,4))
visits.iloc[6, 4] = 'X'
direction_looking

in_loop = False

row_index = 6
col_index = 4

found_escape = False
while (found_escape == False) & (in_loop == False):
    if direction_looking == 'up':
        serie_to_walk = df.iloc[:row_index, col_index]
        try:
            first_stop = max(np.nonzero(serie_to_walk.isin(["#"]))[0])
            if max(visits_facing_up.iloc[first_stop+1:row_index, col_index].str.find("X")) > -1:
                in_loop = True
                break
            visits_facing_up.iloc[first_stop+1:row_index, col_index] = 'X'
            visits.iloc[first_stop+1:row_index, col_index] = 'X'
            direction_looking = "right"
            row_index = first_stop + 1
        except: # no boundary
            visits.iloc[:row_index, col_index] = 'X'
            found_escape = True

    if direction_looking == "right":
        serie_to_walk = df.iloc[row_index, col_index:]
        try:
            first_stop = min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + col_index
            if max(visits_facing_right.iloc[row_index, col_index+1:first_stop].str.find("X")) > -1:
                in_loop = True
                break
            visits_facing_right.iloc[row_index, col_index:first_stop] = 'X'
            visits.iloc[row_index, col_index:first_stop] = 'X'
            direction_looking = "down"
            col_index = first_stop - 1
        except:
            visits.iloc[row_index, col_index:] = 'X'
            found_escape = True
    
    if direction_looking == "down":
        serie_to_walk = df.iloc[row_index:, col_index]
        try:
            first_stop =  min(np.nonzero(serie_to_walk.isin(["#"]))[0]) + row_index
            if max(visits_facing_down.iloc[row_index+1:first_stop, col_index].str.find("X")) > -1:
                in_loop = True
                break
            visits_facing_down.iloc[row_index:first_stop, col_index] = 'X'
            visits.iloc[row_index:first_stop, col_index] = 'X'
            direction_looking = "left"
            row_index = first_stop - 1
        except:
            visits.iloc[row_index:, col_index] = 'X'
            found_escape = True
    
    if direction_looking == "left":
        serie_to_walk = df.iloc[row_index, :col_index]
        try:
            first_stop =  max(np.nonzero(serie_to_walk.isin(["#"]))[0]) 
            if max(visits_facing_left.iloc[row_index, first_stop+1:col_index].str.find("X")) > -1:
                in_loop = True
                break
            visits_facing_left.iloc[row_index, first_stop+1:col_index] = 'X'
            visits.iloc[row_index, first_stop+1:col_index] = 'X'
            direction_looking = "up"
            col_index = first_stop + 1
        except:
            visits.iloc[row_index, :col_index] = 'X'
            found_escape = True


print(row_index)
print(col_index)
print(direction_looking)
display(visits_facing_up)

6
4
up


,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1,2,3,4,5,6,7,8,9,10
0,.,.,.,.,#,.,.,.,.,.
1,.,.,.,.,.,.,.,.,.,#
2,.,.,.,.,.,.,.,.,.,.
3,.,.,#,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,#,.,.
5,.,.,.,.,.,.,.,.,.,.
6,.,#,.,.,^,.,.,.,.,.
7,.,.,.,.,.,.,.,.,#,.
8,#,.,.,.,.,.,.,.,.,.
9,.,.,.,.,.,.,#,.,.,.
